In [36]:
'''
Graficar serie de tiempo por dia

'''

'\nGraficar serie de tiempo por dia\n\n'

In [37]:
#%matplotlib qt5
#%matplotlib inline
#para hacer el grafico animado
%matplotlib qt5

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns; sns.set()

import pandas as pd
import numpy as np


import datetime
from calendar import monthrange #para saber cuantos dias tiene el mes

import ipynb.fs.defs.my_funcs_clusters as myfunc


In [38]:
rutaxls = 'C:/Users/mungu/Documents/DatosWTG.xlsx'
rutaxlsCPot = 'C:/Users/mungu/Dropbox/Doctorado/Python/aero/Curva de potencia vestas 90.xlsx'
#rutaxls = 'C:/Users/mungu/Documents/DatosWTG.xlsx'
#rutaxlsCPot = 'Curva de potencia vestas 90.xlsx'

dataVPxls = pd.read_excel(rutaxls,usecols=[0,1,2],index_col=0,names=['PCTimeStamp','vViento','Pacw'])
dfCPot = pd.read_excel(rutaxlsCPot,usecols=[0,2],index_col=0,names=['vel','pw'])

#marcando los datos faltantes asignando un nan a la fila completa
datamk = dataVPxls
datamk.loc[datamk.isnull().any(axis=1), :] = np.nan

#eliminando filas con NaN. Si busco la fecha anterior debe aparecer error.
cleanData = datamk.dropna()

###calculando la potencia
#la pontencia del archivo de excel es la densidad de potencia acumulada.
#Se tiene que hacer la resta de la potencia siguiente a la anterio y multiplicar por pi*r^2
#el radio es 45m
#si se hace la resta con un array de numpy (values) si se puede restar 
#pues se hace elemento por elemento
dataVP = cleanData.drop('Pacw',axis=1)
dataVP['Pw']= (cleanData.iloc[1:,1].values-cleanData.iloc[0:-1,1]) * np.pi*45**2

#eliminar el ultimo valor pues es NaN
dataVP.drop(dataVP.tail(1).index,inplace=True) 

#Eliminando outliers
dataVP.drop([pd.to_datetime('2016-03-07 09:50:00')],inplace=True)

#leyendo la direccion del viento
dataDirVxls = pd.read_excel(rutaxls,sheet_name=1, usecols=[0,1],index_col=0,names=['PCTimeStamp','Dir'])
#limpiando datos
#marcando los datos faltantes asignando un nan a la fila completa
datamkdir = dataDirVxls
datamkdir.loc[datamkdir.isnull().any(axis=1), :] = np.nan
#eliminando filas con NaN. Si busco la fecha anterior debe aparecer error.
#tambien elimino el ultimo valor como lo hice en los datos de v y p
#el copy es para que no me de la copy warning
dataDir = datamkdir.dropna().copy()
dataDir.drop(dataDir.tail(1).index,inplace=True) 
#eliminando del outlier de viento misma fecha que el de la pontencia
dataDir.drop([pd.to_datetime('2016-03-07 09:50:00')],inplace=True)

direcvrad= np.deg2rad(dataDir['Dir'].values)
velocidades = dataVP.iloc[:]['vViento'].values
vecVel = [-np.sin(direcvrad)*velocidades,np.cos(direcvrad)*velocidades]
vecVelnp=np.array(vecVel).transpose()
#original sin timestamp
#dfVecVel = pd.DataFrame(data=vecVelnp,columns=['vx','vy']
#con timestamp
dfVecVel = pd.DataFrame(data=vecVelnp,columns=['vx','vy'],index=dataVP.index)

In [39]:

f_ini=datetime.datetime(2016, 1, 5,0,0)

f_fin=datetime.datetime(2016, 1, 5,23,59)
 
print(dataVP.loc[f_ini:f_fin])

                     vViento            Pw
PCTimeStamp                               
2016-01-05 00:00:00     16.6  2.061199e+06
2016-01-05 00:10:00     18.1  2.023029e+06
2016-01-05 00:20:00     17.4  1.946688e+06
2016-01-05 00:30:00     20.1  1.959411e+06
2016-01-05 00:40:00     19.1  2.010305e+06
...                      ...           ...
2016-01-05 23:10:00     12.9  1.997582e+06
2016-01-05 23:20:00     13.5  2.010305e+06
2016-01-05 23:30:00     14.0  1.844900e+06
2016-01-05 23:40:00     12.4  1.870347e+06
2016-01-05 23:50:00     12.6  2.035752e+06

[144 rows x 2 columns]


In [40]:
# make 3d axes
fig = plt.figure()
ax = plt.axes(projection='3d') 
# test data
x =dataVP.loc[f_ini:f_fin].vViento.values
y = range(len(x))
z = 1


# plot test data
ax.plot(x, y, z,linewidth=1)


# make labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=-90,azim=2)
# make labels
ax.set_xlabel('Velocidad')
ax.set_ylabel('Hora')
ax.set_zlabel('Dia')
plt.xticks(np.arange(0, max(dataVP.vViento.values)+1, 1.0))
hticks = np.arange(0,145,6)
hval = l/6
plt.yticks(hticks,hval.astype(int),rotation=45)

plt.yticks()
plt.show()
plt.tight_layout()






C:\Users\mungu\AppData\Local\Temp/ipykernel_49756/3556777336.py:30: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  plt.tight_layout()


In [41]:
l/6

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.])

# Gráfica de todos los días (viento)

In [42]:
#RANGO DE FECHAS (un año exacto, 360 días, por fuerza, para que poner mas dias?)
f_ini=datetime.datetime(2016, 1, 1,0,0)
f_fin = f_ini +datetime.timedelta(days=1)
fig = plt.figure()
ax = plt.axes(projection='3d') 
  
for i in range(359):
    
    # make 3d axes
    x =dataVP.loc[f_ini:f_fin].vViento.values
    y = range(len(x))
    z = i
    f_ini=f_fin
    f_fin += datetime.timedelta(days=1)

    # plot test data
    ax.plot(x, y, z,linewidth=1)

ax.view_init(elev=-90,azim=2)
# make labels
ax.set_xlabel('Velocidad')
ax.set_ylabel('Hora')
ax.set_zlabel('Dia')
plt.xticks(np.arange(0, max(dataVP.vViento.values)+1, 1.0))
hticks = np.arange(0,145,6)
hval = l/6
plt.yticks(hticks,hval.astype(int),rotation=45)

plt.yticks()
plt.show()
plt.tight_layout()



C:\Users\mungu\AppData\Local\Temp/ipykernel_49756/3076359811.py:31: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  plt.tight_layout()


# Gráfica mes por mes (viento)

In [48]:
#mes a graficar
mes= 9
anio = 2016

#RANGO DE FECHAS (un año exacto, 360 días, por fuerza, para que poner mas dias?)
f_ini=datetime.datetime(anio, mes, 1,0,0)
f_fin = f_ini +datetime.timedelta(days=1)
fig = plt.figure()
ax = plt.axes(projection='3d') 
#dias en el mes
num_dias = monthrange(anio,mes)[1]
for i in range(num_dias):
    
    # make 3d axes
    x =dataVP.loc[f_ini:f_fin].vViento.values
    y = range(len(x))
    z = i
    f_ini=f_fin
    f_fin += datetime.timedelta(days=1)

    # plot test data
    ax.plot(x, y, z,linewidth=1)
    
ax.view_init(elev=-90,azim=2)
# make labels
ax.set_xlabel('Velocidad')
ax.set_ylabel('Hora')
ax.set_zlabel('Dia')
plt.xticks(np.arange(0, max(dataVP.vViento.values)+1, 1.0))
hticks = np.arange(0,145,6)
hval = l/6
plt.yticks(hticks,hval.astype(int),rotation=45)

plt.yticks()
plt.show()
plt.tight_layout()



C:\Users\mungu\AppData\Local\Temp/ipykernel_49756/63167498.py:37: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  plt.tight_layout()


In [44]:
np.arange(0, max(x)+1, 1.0)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13.])